# <span style="color:red">This is a work in progress and in no way a complete example of using DPV</span>

## Background

FOIP ACT is the Freedom of Information and Protection of Privacy Act that **public bodies** must obide by in Alberta, Canada.

FOIP covers the handling of personal information.

In FOIP, personal information is defined to be, "recorded information about an identifiable individual. It is information that can identify an individual (for example, name, home address, home phone number, e-mail address, ID numbers), and information about an individual (for example, physical description, educational qualifications, blood type)."

It should be noted that, "the definition of personal information does not include information about a sole proprietorship, partnership, unincorporated association or corporation."

The mapping of data handling policy is mapped according to "FOIP Guidelines and Practices: 2009 Edition Chapter 7: Protection of Privacy" (https://web.archive.org/web/20160615221611/https://www.servicealberta.ca/foip/documents/chapter7.pdf).

## Use Case: Municipal Census

Censuses are conducted by municipalities for the planning of public services.

Under the requires guidelines for set by the Alberta Government in 2019 (https://web.archive.org/web/20190929185839/https://open.alberta.ca/dataset/ebee0c79-a9eb-4bf5-993d-30995a2f7554/resource/61613571-e381-4c4e-9f11-2bfe6823ef81/download/2019-municipal-census-manual.pdf), it suggests following a questionaire similar to this (https://web.archive.org/web/20191218172659/http://www.statcan.gc.ca/eng/statistical-programs/instrument/3901_Q8_V1-eng.pdf).

For our use case, we follow the questionaire in the above paragraph to identify the personal information collected.
We assume that the municipal census is conducted in Alberta, and therefore must abide by the FOIP Act.

### Data Handling Policies

We provide 2 different data handling policies: (1) for internal use that contains non-aggreate data, (2) for external publication which only includes aggregate data.

These 2 data handling policies contain the same data subject(s), data controller, and legal basis.

Differing is the processing, purpose, personal data categories, recipients, and technical organisational measure.

#### Internal Use


#### External Publication


## 1. Importing the DPV

In [1]:
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import RDF, RDFS
g = Graph()

# import DPV
DPV = Namespace("http://w3.org/ns/dpv#")
# g.parse('https://www.w3.org/ns/dpv.ttl', format='turtle')

## 2. Creating Personal Data Handling Policy

In [2]:
# Personal Data Handling Policy
# For the use case we consider an internal and external data handling policy

policy = Namespace("http://example.org/policy/")

# Create internal policy
g.add( (policy.Internal, RDF.type, DPV.PersonalDataHandling) )

# Create external policy
g.add( (policy.External, RDF.type, DPV.PersonalDataHandling) )

## 3. Assigning a Data Subject

In [3]:
# Data Subject
# According to FOIP, the data subject will be the person that has personal information collected by a public body
# Note that there should be many more data subjects (as more than one persons' data is collected),
#    but we stick with using 'Bob' for this example.

# create a people namespace, with Bob
person = Namespace("http://example.org/people/")
g.add( (person.bob, RDF.type, DPV.DataSubject) )

# Add to the policies
g.add( (policy.Internal, DPV.hasDataSubject, person.bob) )
g.add( (policy.External, DPV.hasDataSubject, person.bob) )

## 4. Assigning a Data Controller

In [4]:
# Data Controller
# From DPV: The class of Data Controllers that control this particular data handling, 
#     any legal entity that is defined by article 4.7 of GDPR.
#
# In the context of a municipal census, under FOIP, the data controller would be the public body 
#     which is the municipality

# create an organization namespace, with City of Edmonton
org = Namespace("http://example.org/organization/")
g.add( (org.Edmonton, RDF.type, DPV.DataController) )

# Add to the policies
g.add( (policy.Internal, DPV.hasDataController, org.Edmonton) )
g.add( (policy.External, DPV.hasDataController, org.Edmonton) )

## 5. Assigning a Legal Basis

In [5]:
# Legal Basis
# From DPV: A particular legal Basis, which permits personal Data handling (e.g., Consent, etc.)

# There is no ontology for FOIP, we create a placeholder
legal = Namespace("http://example.org/legal/")
g.add( (legal.FOIP, RDF.type, DPV.Consent) )

# In the census there is an option to make data available 100+ years in the future
# Under FOIP, if you agree to release information for public use then it is allowed
# Assume that https://www.servicealberta.ca/foip/legislation/foip-act.cfm is the consent notice
g.add( (legal.FOIP, DPV.consentNotice, URIRef('https://www.servicealberta.ca/foip/legislation/foip-act.cfm')) )

# Add to the policies
g.add( (policy.Internal, DPV.hasLegalBasis, legal.FOIP) )
g.add( (policy.External, DPV.hasLegalBasis, legal.FOIP) )

## 6. Determining Purposes

In [6]:
# Purpose

# Namespace for Purpose
purpose = Namespace("http://example.org/purpose/")

# Internal
g.add( (purpose.CitizenStatistics, RDFS.subClassOf,DPV.ServiceProvision) )
g.add( (purpose.CitizenStatistics, RDFS.subClassOf,DPV.ServiceOptimization) )
g.add( (purpose.CitizenStatistics, RDFS.subClassOf,DPV.ResearchAndDevelopment) )
g.add( (purpose.CitizenStatistics, RDFS.subClassOf,DPV.OptimisationForController) )
g.add( (purpose.CitizenStatistics, RDFS.subClassOf,DPV.InternalResourceOptimisation) )
g.add( (purpose.CitizenStatistics, RDFS.label, Literal("Citizen Statistics")) )
g.add( (purpose.CitizenStatistics, RDFS.comment, Literal("Citizen statistics to deliver and optimize municipal services")) )

g.add( (policy.Internal, DPV.hasPurpose, purpose.CitizenStatistics) )

# External
# The maintain purpose for the external policy is public education throught insights, but there is no such class.
# Most closely related is dpv:SellInsightsFromData, even though the insights provided are free
#    e.g. https://www.edmonton.ca/city_government/facts_figures/municipal-census-results.aspx
# Perhaps the 'Sell' purposes should be more generalized?
g.add( (purpose.PublicCitizenStatistics, RDFS.subClassOf, DPV.SellInsightsFromData) )
g.add( (purpose.PublicCitizenStatistics, RDFS.label, Literal("Public Citizen Statistics")) )
g.add( (purpose.PublicCitizenStatistics, RDFS.comment, Literal("Provide aggregate statistics on municipal census data")) )

g.add( (policy.External, DPV.hasPurpose, purpose.CitizenStatistics) )

## 7. Determining how Data Is Processed

In [7]:
# Processing

# Use the top-level classes of DPV: Disclose, Copy, Obtain, Remove, Store, Transfer, and Transform
# You can also define more specific processing categories for DPV

# Internal
g.add( (policy.Internal, DPV.hasProcessing, DPV.Disclose) )
g.add( (policy.Internal, DPV.hasProcessing, DPV.Copy) )
g.add( (policy.Internal, DPV.hasProcessing, DPV.Obtain) )
g.add( (policy.Internal, DPV.hasProcessing, DPV.Remove) )
g.add( (policy.Internal, DPV.hasProcessing, DPV.Store) )
g.add( (policy.Internal, DPV.hasProcessing, DPV.Transfer) )
g.add( (policy.Internal, DPV.hasProcessing, DPV.Transform) )

# External
g.add( (policy.External, DPV.hasProcessing, DPV.Disclose) )
g.add( (policy.External, DPV.hasProcessing, DPV.Transform) )

## 8. Determining Personal Data Categories

In [8]:
# Personal Data Categories

# Personal data collected is based on
#     https://web.archive.org/web/20191218172659/http://www.statcan.gc.ca/eng/statistical-programs/instrument/3901_Q8_V1-eng.pdf

# Internal
# missing mode of transportation, date of birth, country of origin
# has some generalized categories
g.add( (policy.Internal, DPV.hasPersonalDataCategory, DPV.HouseOwned) ) # residence
g.add( (policy.Internal, DPV.hasPersonalDataCategory, DPV.LifeHistory) ) # place of origin, work history, birth date
g.add( (policy.Internal, DPV.hasPersonalDataCategory, DPV.Family) ) # individual’s family and relationships, marital status
g.add( (policy.Internal, DPV.hasPersonalDataCategory, DPV.Professional) ) # individual’s educational or professional career
g.add( (policy.Internal, DPV.hasPersonalDataCategory, DPV.Contact) ) # telephone number, email, address
g.add( (policy.Internal, DPV.hasPersonalDataCategory, DPV.MedicalHealth) ) # disability, heath condition
g.add( (policy.Internal, DPV.hasPersonalDataCategory, DPV.Demographic) )
g.add( (policy.Internal, DPV.hasPersonalDataCategory, DPV.Name) )
g.add( (policy.Internal, DPV.hasPersonalDataCategory, DPV.Age) )
g.add( (policy.Internal, DPV.hasPersonalDataCategory, DPV.Language) )
g.add( (policy.Internal, DPV.hasPersonalDataCategory, DPV.Gender) )

# External
# In the external census, not all information collected is released
# Example of information released: https://www.edmonton.ca/city_government/facts_figures/municipal-census-results.aspx
g.add( (policy.External, DPV.hasPersonalDataCategory, DPV.Age) )
g.add( (policy.External, DPV.hasPersonalDataCategory, DPV.Gender) )
g.add( (policy.External, DPV.hasPersonalDataCategory, DPV.HouseOwned) )

## 9. Determining Recipients

In [9]:
# Recipient
# From DPV: The entities that can access the result of a data handling action/processing, 
#     any legal entity that is defined by article 4.9 of GDPR, which states - 'recipient' means a natural 
#     or legal person, public authority, agency or another body, to which the personal data are disclosed, 
#     whether a third party or not.

# Internal
# Data is meant for organizational use
g.add( (org.Edmonton, RDF.type, DPV.Recipient) ) # Both controller and recipient
g.add( (policy.Internal, DPV.hasRecipient, org.Edmonton) )

# External
# Assume that General Public is a body and therefore a recipient
g.add( (org.GeneralPublic, RDF.type, DPV.Recipient) )
g.add( (policy.External, DPV.hasRecipient, org.GeneralPublic) )

## 10. Technical Organisational Measures

In [10]:
# Technical Organisational Measures
# From DPV: Technical and organisational measures, for instance security measure, 
#      storage restrictions etc. required/followed when processing data of the declared category

# Internal
g.add( (policy.Internal, DPV.hasTechnicalOrganisationalMeasure, DPV.AuthorisationProcedure) )
g.add( (policy.Internal, DPV.hasTechnicalOrganisationalMeasure, DPV.CodeOfConduct) )
g.add( (policy.Internal, DPV.hasTechnicalOrganisationalMeasure, DPV.Contract) )
g.add( (policy.Internal, DPV.hasTechnicalOrganisationalMeasure, DPV.NDA) )
g.add( (policy.Internal, DPV.hasTechnicalOrganisationalMeasure, DPV.StaffTraining) )

# External
# Privacy is by deisgn as data is aggregated before release
g.add( (policy.External, DPV.hasTechnicalOrganisationalMeasure, DPV.PrivacyByDesign) )

In [11]:
# Export data handling policies to a graph

g.serialize("dpv-foip.ttl", format="turtle")

In [12]:
# How to query with SPARQL
# qres = g.query(
#     """
#     SELECT ?policy
#        WHERE {
#           ?policy a dpv:PersonalDataHandling .
#        }
#     """)

# for row in qres:
#     print(row)